# Import Packages

In [1]:
# pip install sdv

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import time
from sdv.tabular import GaussianCopula, CTGAN, CopulaGAN, TVAE
from sdv.constraints import Inequality, create_custom_constraint
from copy import deepcopy
import warnings
from itertools import combinations
from sd_generator import SD_generator

In [3]:
warnings.filterwarnings("ignore")

# Prepare Data

In [4]:
data = pd.read_csv('data.csv')
data.drop(columns=['Name'], axis=1, inplace=True)
data.head()

,Date_of_birth,Opening_date,Expiry_date,Credit_limit,Statement_balance,Available_credit
0,1949-08-17,2008-09-02,2013-09-02,38000.0,29297.69,8702.31
1,1938-07-04,2019-09-08,2024-09-08,19500.0,15628.50,3871.50
2,1927-03-28,2019-04-23,2024-04-23,11500.0,7388.34,4111.66
3,1936-01-17,2021-10-27,2026-10-27,68500.0,32060.39,36439.61
4,1960-09-05,2010-10-02,2015-10-02,72000.0,56148.13,15851.87


In [5]:
data['Opening'] = data['Opening_date'].apply(lambda x: x.replace('-', ''))
data['Expiry'] = data['Expiry_date'].apply(lambda x: x.replace('-', ''))
data['Client_id'] = data['Opening'] + data['Expiry']
data['Client_id'] = data['Client_id'].apply(lambda x: int(x))
data['Opening'] = data['Opening'].apply(lambda x: int(x))
data['Expiry'] = data['Expiry'].apply(lambda x: int(x))

data = data[['Client_id', 'Date_of_birth', 'Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit', 'Opening', 'Expiry']]

data.head()

,Client_id,Date_of_birth,Opening_date,Expiry_date,Credit_limit,Statement_balance,Available_credit,Opening,Expiry
0,2008090220130902,1949-08-17,2008-09-02,2013-09-02,38000.0,29297.69,8702.31,20080902,20130902
1,2019090820240908,1938-07-04,2019-09-08,2024-09-08,19500.0,15628.50,3871.50,20190908,20240908
2,2019042320240423,1927-03-28,2019-04-23,2024-04-23,11500.0,7388.34,4111.66,20190423,20240423
3,2021102720261027,1936-01-17,2021-10-27,2026-10-27,68500.0,32060.39,36439.61,20211027,20261027
4,2010100220151002,1960-09-05,2010-10-02,2015-10-02,72000.0,56148.13,15851.87,20101002,20151002


In [6]:
data.dtypes

Client_id              int64
Date_of_birth         object
Opening_date          object
Expiry_date           object
Credit_limit         float64
Statement_balance    float64
Available_credit     float64
Opening                int64
Expiry                 int64
dtype: object

# Compare Synthetic Data Generated with & without Constraints Applied

## Generate Synthetic Data with Constraints

In [7]:
d1 = SD_generator(data, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d1.preprocess()
d1.detect_inequality()
d1.detect_arithmetic_equality()
d1.detect_arithmetic_inequality()
d1.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.1115 seconds
8 relationships detected
Execution Time: 0.6386 seconds
1 relationships detected
Execution Time: 1.3814 seconds
8 relationships detected
Execution Time: 0.6191 seconds
2 relationships detected
Execution Time: 0.4775 seconds


### Original Relationships Detected:

In [8]:
print("Inequality: ", d1.inequality_dict)
print("Arithmetic Equality: ", d1.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d1.arithmetic_inequality_dict)
print("Inclusive: ", d1.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010600000000.0]], 'Opening_date': [['Date_of_birth', 6599.0]], 'Expiry_date': [['Opening_date', 1826.0]], 'Opening': [['Expiry_date', 19961750.0], ['Credit_limit', 19900301.0]], 'Expiry': [['Opening', 49999.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.260000000000218]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Create Models

In [9]:
d1.create_constraints(inequality=True, arithmetic_equality=True, arithmetic_inequality=True, inclusive=True)
d1.apply_model("GaussianCopula")
d1.apply_model("CTGAN")
d1.apply_model("CopulaGAN")
d1.apply_model("TVAE")

Constrainsts created successfully!
Execution Time: 0.0002 seconds
Execution Time for training GaussianCopula: 0.1406 seconds
Execution Time for training CTGAN: 58.5756 seconds
Execution Time for training CopulaGAN: 51.2337 seconds
Execution Time for training TVAE: 17.9403 seconds


### Generate Synthetic Data

In [10]:
with_constraint_GaussianCopula = d1.generate("GaussianCopula", num_rows=2000)
with_constraint_CTGAN = d1.generate("CTGAN", num_rows=2000)
with_constraint_CopulaGAN = d1.generate("CopulaGAN", num_rows=2000)
with_constraint_TVAE = d1.generate("TVAE", num_rows=2000)

Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 8146.17it/s]


Synthetic data generated successfully with GaussianCopula model!
Execution Time: 0.576 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 3214.88it/s]


Synthetic data generated successfully with CTGAN model!
Execution Time: 0.9397 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 2532.82it/s]


Synthetic data generated successfully with CopulaGAN model!
Execution Time: 1.128 seconds


Sampling rows: 100%|██████████████████████| 2000/2000 [00:00<00:00, 6308.91it/s]


Synthetic data generated successfully with TVAE model!
Execution Time: 0.6518 seconds


### Detect Relationships on Synthetic Data

### Synthetic Data Generated using GaussianCopula with constraints

In [11]:
d2 = SD_generator(with_constraint_GaussianCopula, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d2.preprocess()
d2.detect_inequality()
d2.detect_arithmetic_equality()
d2.detect_arithmetic_inequality()
d2.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0016 seconds
9 relationships detected
Execution Time: 0.6362 seconds
1 relationships detected
Execution Time: 1.5526 seconds
9 relationships detected
Execution Time: 0.7642 seconds
2 relationships detected
Execution Time: 0.5133 seconds


### New Relationships Detected:

In [12]:
print("Inequality: ", d2.inequality_dict)
print("Arithmetic Equality: ", d2.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d2.arithmetic_inequality_dict)
print("Inclusive: ", d2.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Credit_limit': ['Date_of_birth', 'Statement_balance', 'Available_credit'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2010632900000000.0]], 'Opening_date': [['Date_of_birth', 6641.937504151385]], 'Expiry_date': [['Opening_date', 1826.610054694138]], 'Credit_limit': [['Date_of_birth', 214.7400801894364], ['Statement_balance', 8.810119369354652], ['Available_credit', 320.1999999999971]], 'Opening': [['Expiry_date', 20065224.0], ['Credit_limit', 19945430.630703878]], 'Expiry': [['Opening', 50092.0]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using CTGAN with constraints

In [13]:
d3 = SD_generator(with_constraint_CTGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d3.preprocess()
d3.detect_inequality()
d3.detect_arithmetic_equality()
d3.detect_arithmetic_inequality()
d3.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0014 seconds
8 relationships detected
Execution Time: 0.615 seconds
1 relationships detected
Execution Time: 1.473 seconds
8 relationships detected
Execution Time: 0.6374 seconds
2 relationships detected
Execution Time: 0.568 seconds


### New Relationships Detected:

In [14]:
print("Inequality: ", d3.inequality_dict)
print("Arithmetic Equality: ", d3.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d3.arithmetic_inequality_dict)
print("Inclusive: ", d3.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000130100000000.0]], 'Opening_date': [['Date_of_birth', 6623.674558040897]], 'Expiry_date': [['Opening_date', 1829.6574020380722]], 'Opening': [['Expiry_date', 19961955.0], ['Credit_limit', 19866855.67]], 'Expiry': [['Opening', 50018.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.259999999998399]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using CopulaGAN with constraints

In [15]:
d4 = SD_generator(with_constraint_CopulaGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d4.preprocess()
d4.detect_inequality()
d4.detect_arithmetic_equality()
d4.detect_arithmetic_inequality()
d4.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0023 seconds
8 relationships detected
Execution Time: 0.6145 seconds
1 relationships detected
Execution Time: 1.4699 seconds
8 relationships detected
Execution Time: 0.6239 seconds
2 relationships detected
Execution Time: 0.6289 seconds


### New Relationships Detected:

In [16]:
print("Inequality: ", d4.inequality_dict)
print("Arithmetic Equality: ", d4.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d4.arithmetic_inequality_dict)
print("Inclusive: ", d4.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000097600000000.0]], 'Opening_date': [['Date_of_birth', 6614.884792434292]], 'Expiry_date': [['Opening_date', 1828.4484653696709]], 'Opening': [['Expiry_date', 19961841.0], ['Credit_limit', 19863488.586963013]], 'Expiry': [['Opening', 50001.0]], 'Credit_limit': [['Statement_balance', 5.651505930849453], ['Available_credit', 7.260000000000218]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Synthetic Data Generated using TVAE with constraints

In [17]:
d5 = SD_generator(with_constraint_TVAE, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d5.preprocess()
d5.detect_inequality()
d5.detect_arithmetic_equality()
d5.detect_arithmetic_inequality()
d5.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0025 seconds
8 relationships detected
Execution Time: 0.6242 seconds
1 relationships detected
Execution Time: 1.5008 seconds
8 relationships detected
Execution Time: 0.6316 seconds
2 relationships detected
Execution Time: 0.5802 seconds


### New Relationships Detected:

In [18]:
print("Inequality: ", d5.inequality_dict)
print("Arithmetic Equality: ", d5.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d5.arithmetic_inequality_dict)
print("Inclusive: ", d5.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000070400000000.0]], 'Opening_date': [['Date_of_birth', 6604.906090927572]], 'Expiry_date': [['Opening_date', 1833.9166543924803]], 'Opening': [['Expiry_date', 19961831.0], ['Credit_limit', 19911932.429025795]], 'Expiry': [['Opening', 50024.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.259999999994761]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


## Generate Synthetic Data without Constraints

In [19]:
d6 = SD_generator(data, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d6.preprocess()
d6.detect_inequality()
d6.detect_arithmetic_equality()
d6.detect_arithmetic_inequality()
d6.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0022 seconds
8 relationships detected
Execution Time: 0.6373 seconds
1 relationships detected
Execution Time: 1.5144 seconds
8 relationships detected
Execution Time: 0.6435 seconds
2 relationships detected
Execution Time: 0.4941 seconds


### Original Relationships Detected:

In [20]:
print("Inequality: ", d6.inequality_dict)
print("Arithmetic Equality: ", d6.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d6.arithmetic_inequality_dict)
print("Inclusive: ", d6.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit'], 'Expiry': ['Opening'], 'Credit_limit': ['Statement_balance', 'Available_credit']}
Arithmetic Equality:  {'Credit_limit': [['Statement_balance', 'Available_credit']]}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010600000000.0]], 'Opening_date': [['Date_of_birth', 6599.0]], 'Expiry_date': [['Opening_date', 1826.0]], 'Opening': [['Expiry_date', 19961750.0], ['Credit_limit', 19900301.0]], 'Expiry': [['Opening', 49999.0]], 'Credit_limit': [['Statement_balance', 3.9400000000023283], ['Available_credit', 7.260000000000218]]}
Inclusive:  {'Client_id': [['Opening', 0], ['Expiry', 8]]}


### Create Models

In [21]:
d6.create_constraints(inequality=False, arithmetic_equality=False, arithmetic_inequality=False, inclusive=False)
d6.apply_model("GaussianCopula")
d6.apply_model("CTGAN")
d6.apply_model("CopulaGAN")
d6.apply_model("TVAE")

Constrainsts created successfully!
Execution Time: 0.0 seconds
Execution Time for training GaussianCopula: 0.1665 seconds
Execution Time for training CTGAN: 50.0087 seconds
Execution Time for training CopulaGAN: 52.5914 seconds
Execution Time for training TVAE: 18.1078 seconds


### Generate Synthetic Data

In [22]:
without_constraint_GaussianCopula = d6.generate("GaussianCopula", num_rows=2000)
without_constraint_CTGAN = d6.generate("CTGAN", num_rows=2000)
without_constraint_CopulaGAN = d6.generate("CopulaGAN", num_rows=2000)
without_constraint_TVAE = d6.generate("TVAE", num_rows=2000)

Synthetic data generated successfully with GaussianCopula model!
Execution Time: 0.0743 seconds
Synthetic data generated successfully with CTGAN model!
Execution Time: 0.1142 seconds
Synthetic data generated successfully with CopulaGAN model!
Execution Time: 0.1599 seconds
Synthetic data generated successfully with TVAE model!
Execution Time: 0.0939 seconds


### Detect Relationships on Synthetic Data

### Synthetic Data Generated using GaussianCopula without constraints

In [23]:
d7 = SD_generator(without_constraint_GaussianCopula, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d7.preprocess()
d7.detect_inequality()
d7.detect_arithmetic_equality()
d7.detect_arithmetic_inequality()
d7.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0025 seconds
8 relationships detected
Execution Time: 0.6721 seconds
0 relationships detected
Execution Time: 1.4483 seconds
8 relationships detected
Execution Time: 0.6456 seconds
0 relationships detected
Execution Time: 0.4069 seconds


### New Relationships detected:

In [24]:
print("Inequality: ", d7.inequality_dict)
print("Arithmetic Equality: ", d7.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d7.arithmetic_inequality_dict)
print("Inclusive: ", d7.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000066121617119.0]], 'Opening_date': [['Date_of_birth', 640.0]], 'Expiry_date': [['Opening_date', 1819.0]], 'Opening': [['Expiry_date', 19961956.0], ['Credit_limit', 19901594.0], ['Statement_balance', 19913302.45], ['Available_credit', 19904571.379624713]], 'Expiry': [['Opening', 49029.0]]}
Inclusive:  {}


### Synthetic Data Generated using CTGAN without constraints

In [25]:
d8 = SD_generator(without_constraint_CTGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d8.preprocess()
d8.detect_inequality()
d8.detect_arithmetic_equality()
d8.detect_arithmetic_inequality()
d8.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.002 seconds
14 relationships detected
Execution Time: 0.6198 seconds
0 relationships detected
Execution Time: 1.3097 seconds
14 relationships detected
Execution Time: 1.0925 seconds
0 relationships detected
Execution Time: 0.5104 seconds


### New Relationships detected:

In [26]:
print("Inequality: ", d8.inequality_dict)
print("Arithmetic Equality: ", d8.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d8.arithmetic_inequality_dict)
print("Inclusive: ", d8.inclusive_dict)

Inequality:  {'Client_id': ['Opening', 'Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Date_of_birth'], 'Opening': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Opening', 2000010599872296.0], ['Expiry', 2000010599786523.0]], 'Opening_date': [['Date_of_birth', 233.0]], 'Expiry_date': [['Date_of_birth', 876.0]], 'Opening': [['Opening_date', 19955183.0], ['Expiry_date', 19953357.0], ['Credit_limit', 19900106.0], ['Statement_balance', 19901941.96], ['Available_credit', 19905403.86406966]], 'Expiry': [['Opening_date', 20005281.0], ['Expiry_date', 20004381.0], ['Credit_limit', 19950106.0], ['Statement_balance', 19958823.93], ['Available_credit', 19951337.67]]}
Inclusive:  {}


### Synthetic Data Generated using CopulaGAN without constraints

In [27]:
d9 = SD_generator(without_constraint_CopulaGAN, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d9.preprocess()
d9.detect_inequality()
d9.detect_arithmetic_equality()
d9.detect_arithmetic_inequality()
d9.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0026 seconds
14 relationships detected
Execution Time: 0.6254 seconds
0 relationships detected
Execution Time: 1.2708 seconds
14 relationships detected
Execution Time: 1.0885 seconds
0 relationships detected
Execution Time: 0.4846 seconds


### New Relationships detected:

In [28]:
print("Inequality: ", d9.inequality_dict)
print("Arithmetic Equality: ", d9.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d9.arithmetic_inequality_dict)
print("Inclusive: ", d9.inclusive_dict)

Inequality:  {'Client_id': ['Opening', 'Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Date_of_birth'], 'Opening': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Opening', 2000010637892355.0], ['Expiry', 2000010637973354.0]], 'Opening_date': [['Date_of_birth', 178.0]], 'Expiry_date': [['Date_of_birth', 1137.0]], 'Opening': [['Opening_date', 19955412.0], ['Expiry_date', 19953391.0], ['Credit_limit', 19900396.0], ['Statement_balance', 19903411.65], ['Available_credit', 19905776.993194383]], 'Expiry': [['Opening_date', 20005434.0], ['Expiry_date', 20004105.0], ['Credit_limit', 19950163.0], ['Statement_balance', 19952851.66], ['Available_credit', 19959151.69066049]]}
Inclusive:  {}


### Synthetic Data Generated using TVAE without constraints

In [29]:
d10 = SD_generator(without_constraint_TVAE, inequality_threshold=0.95, arithmetic_equality_threshold=0.95, inclusive_threshold=1.0)
d10.preprocess()
d10.detect_inequality()
d10.detect_arithmetic_equality()
d10.detect_arithmetic_inequality()
d10.detect_inclusive()

Date types reformatted and missing values handled successfully!
Execution Time: 0.0039 seconds
8 relationships detected
Execution Time: 0.5994 seconds
0 relationships detected
Execution Time: 1.4006 seconds
8 relationships detected
Execution Time: 0.6267 seconds
0 relationships detected
Execution Time: 0.488 seconds


### New Relationships detected:

In [30]:
print("Inequality: ", d10.inequality_dict)
print("Arithmetic Equality: ", d10.arithmetic_equality_dict)
print("Arithmetic Inequality: ", d10.arithmetic_inequality_dict)
print("Inclusive: ", d10.inclusive_dict)

Inequality:  {'Client_id': ['Expiry'], 'Opening_date': ['Date_of_birth'], 'Expiry_date': ['Opening_date'], 'Opening': ['Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'], 'Expiry': ['Opening']}
Arithmetic Equality:  {}
Arithmetic Inequality:  {'Client_id': [['Expiry', 2000010599998041.0]], 'Opening_date': [['Date_of_birth', 1494.0]], 'Expiry_date': [['Opening_date', 44.0]], 'Opening': [['Expiry_date', 19960755.0], ['Credit_limit', 19900106.0], ['Statement_balance', 19910671.5], ['Available_credit', 19928130.461960886]], 'Expiry': [['Opening', 1636.0]]}
Inclusive:  {}
